# Information Retrieval 1#
## Assignment 2: Retrieval models [100 points] ##

In this assignment you will get familiar with basic and advanced information retrieval concepts. You will implement different information retrieval ranking models and evaluate their performance.

We provide you with a Indri index. To query the index, you'll use a Python package ([pyndri](https://github.com/cvangysel/pyndri)) that allows easy access to the underlying document statistics.

For evaluation you'll use the [TREC Eval](https://github.com/usnistgov/trec_eval) utility, provided by the National Institute of Standards and Technology of the United States. TREC Eval is the de facto standard way to compute Information Retrieval measures and is frequently referenced in scientific papers.

This is a **groups-of-three assignment**, the deadline is **Wednesday, January 31st**. Code quality, informative comments and convincing analysis of the results will be considered when grading. Submission should be done through blackboard, questions can be asked on the course [Piazza](piazza.com/university_of_amsterdam/spring2018/52041inr6y/home).

### Technicalities (must-read!) ###

The assignment directory is organized as follows:
   * `./assignment.ipynb` (this file): the description of the assignment.
   * `./index/`: the index we prepared for you.
   * `./ap_88_90/`: directory with ground-truth and evaluation sets:
      * `qrel_test`: test query relevance collection (**test set**).
      * `qrel_validation`: validation query relevance collection (**validation set**).
      * `topics_title`: semicolon-separated file with query identifiers and terms.

You will need the following software packages (tested with Python 3.5 inside [Anaconda](https://conda.io/docs/user-guide/install/index.html)):
   * Python 3.5 and Jupyter
   * Indri + Pyndri (Follow the installation instructions [here](https://github.com/nickvosk/pyndri/blob/master/README.md))
   * gensim [link](https://radimrehurek.com/gensim/install.html)
   * TREC Eval [link](https://github.com/usnistgov/trec_eval)

### TREC Eval primer ###
The TREC Eval utility can be downloaded and compiled as follows:

    git clone https://github.com/usnistgov/trec_eval.git
    cd trec_eval
    make

TREC Eval computes evaluation scores given two files: ground-truth information regarding relevant documents, named *query relevance* or *qrel*, and a ranking of documents for a set of queries, referred to as a *run*. The *qrel* will be supplied by us and should not be changed. For every retrieval model (or combinations thereof) you will generate a run of the top-1000 documents for every query. The format of the *run* file is as follows:

    $query_identifier Q0 $document_identifier $rank_of_document_for_query $query_document_similarity $run_identifier
    
where
   * `$query_identifier` is the unique identifier corresponding to a query (usually this follows a sequential numbering).
   * `Q0` is a legacy field that you can ignore.
   * `$document_identifier` corresponds to the unique identifier of a document (e.g., APXXXXXXX where AP denotes the collection and the Xs correspond to a unique numerical identifier).
   * `$rank_of_document_for_query` denotes the rank of the document for the particular query. This field is ignored by TREC Eval and is only maintained for legacy support. The ranks are computed by TREC Eval itself using the `$query_document_similarity` field (see next). However, it remains good practice to correctly compute this field.
   * `$query_document_similarity` is a score indicating the similarity between query and document where a higher score denotes greater similarity.
   * `$run_identifier` is an identifier of the run. This field is for your own convenience and has no purpose beyond bookkeeping.
   
For example, say we have two queries: `Q1` and `Q2` and we rank three documents (`DOC1`, `DOC2`, `DOC3`). For query `Q1`, we find the following similarity scores `score(Q1, DOC1) = 1.0`, `score(Q1, DOC2) = 0.5`, `score(Q1, DOC3) = 0.75`; and for `Q2`: `score(Q2, DOC1) = -0.1`, `score(Q2, DOC2) = 1.25`, `score(Q1, DOC3) = 0.0`. We can generate run using the following snippet:

In [2]:
import logging
import sys
import os
import os.path
import time
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import math
from operator import itemgetter


def write_run(model_name, data, out_f,
              max_objects_per_query=sys.maxsize,
              skip_sorting=False):
    """
    Write a run to an output file.
    Parameters:
        - model_name: identifier of run.
        - data: dictionary mapping topic_id to object_assesments;
            object_assesments is an iterable (list or tuple) of
            (relevance, object_id) pairs.
            The object_assesments iterable is sorted by decreasing order.
        - out_f: output file stream.
        - max_objects_per_query: cut-off for number of objects per query.
    """
    for subject_id, object_assesments in data.items():
        if not object_assesments:
            logging.warning('Received empty ranking for %s; ignoring.',
                            subject_id)

            continue

        # Probe types, to make sure everything goes alright.
        # assert isinstance(object_assesments[0][0], float) or \
        #     isinstance(object_assesments[0][0], np.float32)
        assert isinstance(object_assesments[0][1], str) or \
               isinstance(object_assesments[0][1], bytes)

        if not skip_sorting:
            object_assesments = sorted(object_assesments, reverse=True)

        if max_objects_per_query < sys.maxsize:
            object_assesments = object_assesments[:max_objects_per_query]

        if isinstance(subject_id, bytes):
            subject_id = subject_id.decode('utf8')

        for rank, (relevance, object_id) in enumerate(object_assesments):
            if isinstance(object_id, bytes):
                object_id = object_id.decode('utf8')

            out_f.write(
                '{subject} Q0 {object} {rank} {relevance} '
                '{model_name}\n'.format(
                    subject=subject_id,
                    object=object_id,
                    rank=rank + 1,
                    relevance=relevance,
                    model_name=model_name))


# The following writes the run to standard output.
# In your code, you should write the runs to local
# storage in order to pass them to trec_eval.
write_run(
    model_name='example',
    data={
        'Q1': ((1.0, 'DOC1'), (0.5, 'DOC2'), (0.75, 'DOC3')),
        'Q2': ((-0.1, 'DOC1'), (1.25, 'DOC2'), (0.0, 'DOC3')),
    },
    out_f=sys.stdout,
    max_objects_per_query=1000)

Q1 Q0 DOC1 1 1.0 example
Q1 Q0 DOC3 2 0.75 example
Q1 Q0 DOC2 3 0.5 example
Q2 Q0 DOC2 1 1.25 example
Q2 Q0 DOC3 2 0.0 example
Q2 Q0 DOC1 3 -0.1 example


Now, imagine that we know that `DOC1` is relevant and `DOC3` is non-relevant for `Q1`. In addition, for `Q2` we only know of the relevance of `DOC3`. The query relevance file looks like:

    Q1 0 DOC1 1
    Q1 0 DOC3 0
    Q2 0 DOC3 1
    
We store the run and qrel in files `example.run` and `example.qrel` respectively on disk. We can now use TREC Eval to compute evaluation measures. In this example, we're only interested in Mean Average Precision and we'll only show this below for brevity. However, TREC Eval outputs much more information such as NDCG, recall, precision, etc.

    $ trec_eval -m all_trec -q example.qrel example.run | grep -E "^map\s"
    > map                   	Q1	1.0000
    > map                   	Q2	0.5000
    > map                   	all	0.7500
    
Now that we've discussed the output format of rankings and how you can compute evaluation measures from these rankings, we'll now proceed with an overview of the indexing framework you'll use.

### Pyndri primer ###
For this assignment you will use [Pyndri](https://github.com/cvangysel/pyndri) [[1](https://arxiv.org/abs/1701.00749)], a python interface for [Indri](https://www.lemurproject.org/indri.php). We have indexed the document collection and you can query the index using Pyndri. We will start by giving you some examples of what Pyndri can do:

First we read the document collection index with Pyndri:

In [3]:
import pyndri

index = pyndri.Index('index/')

The loaded index can be used to access a collection of documents in an easy manner. We'll give you some examples to get some idea of what it can do, it is up to you to figure out how to use it for the remainder of the assignment.

First let's look at the number of documents, since Pyndri indexes the documents using incremental identifiers we can simply take the lowest index and the maximum document and consider the difference:

In [4]:
print("There are %d documents in this collection." % (index.maximum_document() - index.document_base()))

There are 164597 documents in this collection.


Let's take the first document out of the collection and take a look at it:

In [5]:
example_document = index.document(index.document_base())
print(example_document)

('AP890425-0001', (1360, 192, 363, 0, 880, 0, 200, 0, 894, 412, 92160, 3, 192, 0, 363, 34, 1441, 0, 174134, 0, 200, 0, 894, 412, 2652, 0, 810, 107, 49, 4903, 420, 0, 1, 48, 35, 489, 0, 35, 687, 192, 243, 0, 249311, 1877, 0, 1651, 1174, 0, 2701, 117, 412, 0, 810, 391, 245233, 1225, 5838, 16, 0, 233156, 3496, 0, 393, 17, 0, 2435, 4819, 930, 0, 0, 200, 0, 894, 0, 22, 398, 145, 0, 3, 271, 115, 0, 1176, 2777, 292, 0, 725, 192, 0, 0, 50046, 0, 1901, 1130, 0, 192, 0, 408, 0, 243779, 0, 0, 553, 192, 0, 363, 0, 3747, 0, 0, 0, 0, 1176, 0, 1239, 0, 0, 1115, 17, 0, 0, 585, 192, 1963, 0, 0, 412, 54356, 0, 773, 0, 0, 0, 192, 0, 0, 1130, 0, 363, 0, 545, 192, 0, 1174, 1901, 1130, 0, 4, 398, 145, 39, 0, 577, 0, 355, 0, 491, 0, 6025, 0, 0, 193156, 88, 34, 437, 0, 0, 1852, 0, 828, 0, 1588, 0, 0, 0, 2615, 0, 0, 107, 49, 420, 0, 0, 190, 7, 714, 2701, 0, 237, 192, 157, 0, 412, 34, 437, 0, 0, 200, 6025, 26, 0, 0, 0, 0, 363, 0, 22, 398, 145, 0, 200, 638, 126222, 6018, 0, 880, 0, 0, 161, 0, 0, 319, 894, 2701, 

Here we see a document consists of two things, a string representing the external document identifier and an integer list representing the identifiers of words that make up the document. Pyndri uses integer representations for words or terms, thus a token_id is an integer that represents a word whereas the token is the actual text of the word/term. Every id has a unique token and vice versa with the exception of stop words: words so common that there are uninformative, all of these receive the zero id.

To see what some ids and their matching tokens we take a look at the dictionary of the index:

In [5]:
token2id, id2token, _ = index.get_dictionary()
print(list(id2token.items())[:15])

[(1, 'new'), (2, 'percent'), (3, 'two'), (4, '1'), (5, 'people'), (6, 'million'), (7, '000'), (8, 'government'), (9, 'president'), (10, 'years'), (11, 'state'), (12, '2'), (13, 'states'), (14, 'three'), (15, 'time')]


Using this dictionary we can see the tokens for the (non-stop) words in our example document:

In [6]:
print([id2token[word_id] for word_id in example_document[1] if word_id > 0])

['52', 'students', 'arrested', 'takeover', 'university', 'massachusetts', 'building', 'fifty', 'two', 'students', 'arrested', 'tuesday', 'evening', 'occupying', 'university', 'massachusetts', 'building', 'overnight', 'protest', 'defense', 'department', 'funded', 'research', 'new', 'york', 'city', 'thousands', 'city', 'college', 'students', 'got', 'unscheduled', 'holiday', 'demonstrators', 'occupied', 'campus', 'administration', 'building', 'protest', 'possible', 'tuition', 'increases', 'prompting', 'officials', 'suspend', 'classes', '60', 'police', 'riot', 'gear', 'arrived', 'university', 'massachusetts', '5', 'p', 'm', 'two', 'hours', 'later', 'bus', 'drove', 'away', '29', 'students', 'camped', 'memorial', 'hall', 'students', 'charged', 'trespassing', '23', 'students', 'arrested', 'lying', 'bus', 'prevent', 'leaving', 'police', '300', 'students', 'stood', 'building', 'chanting', 'looking', 'students', 'hall', 'arrested', '35', 'students', 'occupied', 'memorial', 'hall', '1', 'p', 'm',

The reverse can also be done, say we want to look for news about the "University of Massachusetts", the tokens of that query can be converted to ids using the reverse dictionary:

In [6]:
query_tokens = index.tokenize("University of Massachusetts")
print("Query by tokens:", query_tokens)
query_id_tokens = [token2id.get(query_token,0) for query_token in query_tokens]
print("Query by ids with stopwords:", query_id_tokens)
query_id_tokens = [word_id for word_id in query_id_tokens if word_id > 0]
print("Query by ids without stopwords:", query_id_tokens)

Query by tokens: ['university', '', 'massachusetts']


NameError: name 'token2id' is not defined

Naturally we can now match the document and query in the id space, let's see how often a word from the query occurs in our example document:

In [8]:
matching_words = sum([True for word_id in example_document[1] if word_id in query_id_tokens])
print("Document %s has %d word matches with query: \"%s\"." % (example_document[0], matching_words, ' '.join(query_tokens)))
print("Document %s and query \"%s\" have a %.01f%% overlap." % (example_document[0], ' '.join(query_tokens),matching_words/float(len(example_document[1]))*100))

Document AP890425-0001 has 13 word matches with query: "university  massachusetts".
Document AP890425-0001 and query "university  massachusetts" have a 2.5% overlap.


While this is certainly not everything Pyndri can do, it should give you an idea of how to use it. Please take a look at the [examples](https://github.com/cvangysel/pyndri) as it will help you a lot with this assignment.

**CAUTION**: Avoid printing out the whole index in this Notebook as it will generate a lot of output and is likely to corrupt the Notebook.

### Parsing the query file
You can parse the query file (`ap_88_89/topics_title`) using the following snippet:

In [7]:
import collections
import io
import logging
import sys

def parse_topics(file_or_files,
                 max_topics=sys.maxsize, delimiter=';'):
    assert max_topics >= 0 or max_topics is None

    topics = collections.OrderedDict()

    if not isinstance(file_or_files, list) and \
            not isinstance(file_or_files, tuple):
        if hasattr(file_or_files, '__iter__'):
            file_or_files = list(file_or_files)
        else:
            file_or_files = [file_or_files]

    for f in file_or_files:
        assert isinstance(f, io.IOBase)

        for line in f:
            assert(isinstance(line, str))

            line = line.strip()

            if not line:
                continue

            topic_id, terms = line.split(delimiter, 1)

            if topic_id in topics and (topics[topic_id] != terms):
                    logging.error('Duplicate topic "%s" (%s vs. %s).',
                                  topic_id,
                                  topics[topic_id],
                                  terms)

            topics[topic_id] = terms

            if max_topics > 0 and len(topics) >= max_topics:
                break

    return topics
ordered_dict=[]
with open('./ap_88_89/topics_title', 'r') as f_topics:
      ordered_dict=parse_topics([f_topics])

### Task 1: Implement and compare lexical IR methods [35 points] ### 

In this task you will implement a number of lexical methods for IR using the **Pyndri** framework. Then you will evaluate these methods on the dataset we have provided using **TREC Eval**.

Use the **Pyndri** framework to get statistics of the documents (term frequency, document frequency, collection frequency; **you are not allowed to use the query functionality of Pyndri**) and implement the following scoring methods in **Python**:

- [TF-IDF](http://nlp.stanford.edu/IR-book/html/htmledition/tf-idf-weighting-1.html) and 
- [BM25](http://nlp.stanford.edu/IR-book/html/htmledition/okapi-bm25-a-non-binary-model-1.html) with k1=1.2 and b=0.75. **[5 points]**
- Language models ([survey](https://drive.google.com/file/d/0B-zklbckv9CHc0c3b245UW90NE0/view))
    - Jelinek-Mercer (explore different values of 𝛌 in the range [0.1, 0.5, 0.9]). **[5 points]**
    - Dirichlet Prior (explore different values of 𝛍 [500, 1000, 1500]). **[5 points]**
    - Absolute discounting (explore different values of 𝛅 in the range [0.1, 0.5, 0.9]). **[5 points]**
    - [Positional Language Models](http://sifaka.cs.uiuc.edu/~ylv2/pub/sigir09-plm.pdf) define a language model for each position of a document, and score a document based on the scores of its PLMs. The PLM is estimated based on propagated counts of words within a document through a proximity-based density function, which both captures proximity heuristics and achieves an effect of “soft” passage retrieval. Implement the PLM, all five kernels, but only the Best position strategy to score documents. Use 𝛔 equal to 50, and Dirichlet smoothing with 𝛍 optimized on the validation set (decide how to optimize this value yourself and motivate your decision in the report). **[10 points]**
    
Implement the above methods and report evaluation measures (on the test set) using the hyper parameter values you optimized on the validation set (also report the values of the hyper parameters). Use TREC Eval to obtain the results and report on `NDCG@10`, Mean Average Precision (`MAP@1000`), `Precision@5` and `Recall@1000`.

For the language models, create plots showing `NDCG@10` with varying values of the parameters. You can do this by chaining small scripts using shell scripting (preferred) or execute trec_eval using Python's `subprocess`.

Compute significance of the results using a [two-tailed paired Student t-test](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html) **[5 points]**. Be wary of false rejection of the null hypothesis caused by the [multiple comparisons problem](https://en.wikipedia.org/wiki/Multiple_comparisons_problem). There are multiple ways to mitigate this problem and it is up to you to choose one.

Analyse the results by identifying specific queries where different methods succeed or fail and discuss possible reasons that cause these differences. This is *very important* in order to understand who the different retrieval functions behave.

**NOTE**: Don’t forget to use log computations in your calculations to avoid underflows. 

**IMPORTANT**: You should structure your code around the helper functions we provide below.

In [8]:
with open('./ap_88_89/topics_title', 'r') as f_topics:
    queries = parse_topics([f_topics])

index = pyndri.Index('index/')

num_documents = index.maximum_document() - index.document_base()

dictionary = pyndri.extract_dictionary(index)

tokenized_queries = {
    query_id: [dictionary.translate_token(token)
               for token in index.tokenize(query_string)
               if dictionary.has_token(token)]
    for query_id, query_string in queries.items()}

query_term_ids = set(
    query_term_id
    for query_term_ids in tokenized_queries.values()
    for query_term_id in query_term_ids)

print('Gathering statistics about', len(query_term_ids), 'terms.')

# inverted index creation.

document_lengths = {}
unique_terms_per_document = {}

inverted_index = collections.defaultdict(dict)
collection_frequencies = collections.defaultdict(int)

total_terms = 0

for int_doc_id in range(index.document_base(), index.maximum_document()):
    ext_doc_id, doc_token_ids = index.document(int_doc_id)

    document_bow = collections.Counter(
        token_id for token_id in doc_token_ids
        if token_id > 0)
    document_length = sum(document_bow.values())

    document_lengths[int_doc_id] = document_length
    total_terms += document_length

    unique_terms_per_document[int_doc_id] = len(document_bow)

    for query_term_id in query_term_ids:
        assert query_term_id is not None

        document_term_frequency = document_bow.get(query_term_id, 0)

        if document_term_frequency == 0:
            continue

        collection_frequencies[query_term_id] += document_term_frequency
        inverted_index[query_term_id][int_doc_id] = document_term_frequency

avg_doc_length = total_terms / num_documents
print('Done')

Gathering statistics about 456 terms.
Done


In [108]:
for query_id, query_string in queries.items():     

SyntaxError: unexpected EOF while parsing (<ipython-input-108-4a81b96d021d>, line 1)

In [21]:
-

In [41]:
def tfidf(int_document_id, query_term_id, document_term_freq):
    """
    Scoring function for a document and a query term
    
    :param int_document_id: the document id
    :param query_token_id: the query term id (assuming you have split the query to tokens)
    :param document_term_freq: the document term frequency of the query term 
    """
    # TODO implement the function
    score = 0
    
    document_lenght=document_lengths[int_document_id]
    if(document_lenght==0):
        return 0
    tf=np.log(1+(document_term_freq/document_lenght))
    idf=np.log(num_documents/len(inverted_index[query_term_id]))
    score=tf*idf
    return score


def create_vector_query(query_tokens):
    dic={}
    count=0
    for i in range(0,len(query_tokens)):
        if (query_tokens[i] not in dic):
            dic[query_tokens[i]]=count
            count=count+1
    vector=np.zeros(len(dic))
    for i in range(0,len(query_tokens)):
        index=dic[query_tokens[i]]
        vector[index]=vector[index]+1
    return vector,dic
def create_vector(dic,doc_tokens):
    vector=np.zeros(len(dic))
    for key in doc_tokens.keys():
        index=dic[key]
        vector[index]=doc_tokens[key]
    return vector
def calc_euclidean_vector(vector):
    distance=0
    for i in range(0,len(vector)):
        distance=distance+vector[i]*vector[i]
    return math.sqrt(distance)
def sort_docs_by_score(list_docs):
    list_docs.sort(key=itemgetter(1), reverse=True)
    return list_docs
def create_doc_dict_on_query(query_tokens):
    doc_dict={}
    for token in query_tokens:
        temp_dict=inverted_index[token]
        for doc_id in temp_dict.keys():
            word_count=temp_dict[doc_id]
            if(doc_id not in doc_dict):
                doc_dict[doc_id]={}
            doc_dict[doc_id][token]=word_count
    return doc_dict

# function to calcuate score for all the docs based on query
def get_tf_idf_get_score_doc(query_id,docu_id):
    query_tokens=tokenized_queries[query_id]    
    query_vector,dic=create_vector_query(query_tokens)
    doc_dict={}
    doc_vector=np.zeros(len(dic))
    ind=-1
    count=0
    for token in query_tokens:
        word_count=0
        if(docu_id in inverted_index[token]):
            word_count=inverted_index[token][docu_id]
        ind=0   
        for key_temp in dic.keys():
            if(key_temp==token):
                tf_idf_weight=tfidf(docu_id,token,word_count)
                doc_vector[ind]=tf_idf_weight*word_count
            else:
                ind=ind+1
    query_eucli=calc_euclidean_vector(query_vector)
    doc_eucli=calc_euclidean_vector(doc_vector)
    if(doc_eucli==0):
        return 0
    similarity=cosine_similarity(doc_vector.reshape(1,len(doc_vector)),query_vector.reshape(1,len(doc_vector)))/(doc_eucli*query_eucli)
    
    #scores=sort_docs_by_score(scores)
    return math.log(similarity[0][0])
    

# query _tokens for a given query id
#query_tokens=tokenized_queries['191']    
#tf_idf_scores=get_tf_idf_get_score_doc('191',137864)
#print(tf_idf_scores)
    
# combining the two functions above: 
run_retrieval('tfidf', get_tf_idf_get_score_doc)

# TODO implement the rest of the retrieval functions 

# TODO implement tools to help you with the analysis of the results.

Retrieving using tfidf
Queries completed 0
1966
29
229
230
231
251
329
349
447
491
665
780
917
977
1142
1147
1234
1353
1470
1570
1657
1724
1729
1786
1835
1837
1869
2016
2233
2460
2572
2621
2703
2770
2842
2881
2900
2972
2992
2994
3003
3020
3115
3117
3171
3239
3258
3321
3349
3661
3833
3842
3868
3904
4009
4082
4329
4381
4486
4502
4930
5016
5025
5061
5269
5518
5523
5580
5581
5713
5825
5830
5848
5993
6040
6127
6130
6217
6242
6328
6335
6433
6511
6571
6610
6718
6797
6826
6896
6909
6910
6951
7069
7436
7463
7467
7583
7644
7681
7743
8294
8324
8446
8515
8549
8811
8892
8943
9090
9270
9309
9467
9677
9688
9702
9730
9734
9810
9837
9906
9911
10000
10173
10198
10202
10282
10309
10320
10402
10569
10683
10874
10882
11003
11012
11105
11141
11189
11197
11213
11398
11593
11611
11688
11780
11834
11836
11872
11963
11983
12137
12180
12351
12463
12473
12600
12656
12767
12842
12871
13047
13069
13119
13141
13666
13685
13730
13799
13815
14097
14171
14213
14260
14428
14541
14608
14716
14859
15010
15213
15592
15626


4539
4553
4555
4566
4571
4578
4588
4599
4608
4609
4619
4622
4632
4640
4647
4652
4674
4678
4681
4682
4687
4707
4708
4718
4725
4730
4734
4752
4755
4773
4799
4803
4804
4811
4822
4823
4831
4860
4861
4870
4871
4875
4877
4880
4889
4895
4901
4910
4911
4913
4915
4920
4922
4940
4941
4943
4944
4954
4962
4974
4984
4988
4994
4995
5001
5002
5009
5031
5039
5043
5046
5052
5056
5079
5086
5088
5094
5096
5098
5100
5110
5115
5116
5121
5129
5130
5134
5146
5161
5163
5184
5194
5199
5228
5233
5234
5266
5270
5276
5283
5292
5301
5304
5305
5307
5311
5325
5330
5333
5338
5349
5355
5362
5371
5374
5383
5392
5393
5407
5409
5410
5411
5417
5418
5423
5429
5437
5438
5451
5452
5474
5502
5528
5555
5557
5558
5559
5560
5581
5589
5591
5597
5606
5622
5634
5641
5653
5657
5668
5674
5676
5677
5681
5683
5689
5693
5707
5711
5712
5714
5721
5722
5737
5738
5741
5750
5751
5760
5762
5765
5785
5787
5790
5822
5836
5847
5860
5868
5869
5870
5879
5881
5887
5909
5911
5915
5920
5929
5931
5933
5950
5954
5959
5965
5970
5977
6001
6016
6022
6026


15084
15087
15090
15093
15094
15095
15100
15114
15115
15125
15126
15137
15143
15145
15146
15148
15160
15168
15178
15183
15184
15195
15197
15204
15210
15218
15220
15221
15228
15233
15238
15241
15248
15249
15250
15257
15262
15264
15265
15266
15318
15323
15332
15334
15341
15353
15368
15373
15379
15390
15396
15403
15425
15427
15438
15440
15443
15455
15457
15463
15485
15495
15497
15502
15504
15509
15519
15520
15538
15539
15540
15542
15543
15549
15558
15561
15569
15572
15608
15621
15630
15643
15652
15659
15678
15683
15705
15706
15707
15713
15717
15719
15728
15734
15740
15748
15756
15760
15763
15766
15769
15777
15783
15791
15799
15803
15814
15821
15824
15825
15829
15835
15838
15843
15889
15919
15928
15929
15934
15944
15953
15954
15960
15963
15966
15967
15973
15976
15978
15979
15994
16011
16016
16018
16029
16035
16037
16041
16058
16067
16069
16070
16076
16088
16095
16096
16108
16109
16110
16112
16122
16136
16150
16152
16153
16158
16163
16169
16174
16181
16183
16187
16188
16195
16207
16211
1621

25434
25447
25451
25460
25464
25471
25479
25480
25485
25488
25498
25510
25511
25531
25532
25536
25537
25547
25550
25555
25578
25605
25621
25629
25642
25649
25652
25662
25665
25667
25668
25670
25672
25674
25685
25688
25692
25695
25700
25702
25703
25708
25713
25716
25720
25727
25740
25746
25747
25755
25756
25773
25774
25779
25780
25790
25796
25799
25802
25817
25818
25826
25861
25865
25879
25884
25893
25896
25902
25907
25915
25922
25928
25929
25947
25957
25965
25980
25983
25994
25995
26000
26002
26016
26019
26026
26027
26051
26054
26059
26060
26066
26115
26151
26155
26156
26160
26161
26166
26191
26221
26228
26229
26236
26241
26244
26245
26250
26252
26260
26262
26266
26273
26278
26285
26291
26297
26301
26304
26311
26313
26316
26319
26336
26337
26344
26352
26353
26355
26358
26360
26362
26363
26364
26367
26371
26380
26389
26392
26397
26403
26414
26429
26434
26448
26455
26465
26466
26467
26469
26472
26480
26485
26487
26488
26491
26495
26502
26510
26521
26524
26527
26530
26535
26536
26538
2654

36126
36127
36148
36153
36158
36160
36163
36173
36175
36182
36183
36184
36188
36190
36192
36193
36203
36209
36224
36231
36235
36243
36244
36252
36254
36256
36263
36282
36285
36286
36297
36306
36308
36309
36313
36329
36340
36350
36369
36387
36391
36402
36407
36417
36425
36447
36449
36451
36454
36455
36456
36472
36474
36478
36489
36491
36495
36505
36509
36511
36512
36514
36517
36519
36530
36538
36540
36543
36544
36545
36546
36555
36560
36562
36565
36576
36584
36586
36606
36623
36628
36658
36659
36660
36666
36676
36682
36683
36685
36689
36691
36694
36697
36709
36714
36717
36721
36723
36725
36730
36743
36759
36764
36765
36783
36784
36786
36789
36790
36794
36808
36809
36814
36816
36819
36835
36841
36857
36867
36868
36873
36876
36884
36899
36901
36913
36919
36920
36924
36925
36935
36938
36942
36943
36944
36969
36976
36978
36992
37000
37009
37010
37014
37018
37025
37029
37032
37033
37036
37038
37045
37046
37059
37064
37067
37070
37071
37073
37081
37084
37127
37133
37140
37145
37151
37152
3716

...

155539
155549
155554
155555
155557
155559
155567
155571
155572
155573
155577
155579
155581
155584
155597
155603
155611
155627
155637
155647
155660
155661
155662
155676
155679
155681
155683
155684
155695
155697
155699
155704
155728
155732
155748
155765
155774
155775
155778
155779
155781
155783
155784
155789
155803
155816
155825
155827
155834
155843
155863
155865
155866
155876
155885
155897
155899
155900
155903
155913
155928
155929
155930
155932
155933
155943
155959
156025
156029
156044
156050
156051
156053
156060
156076
156092
156095
156096
156099
156103
156104
156106
156108
156113
156116
156117
156120
156122
156123
156125
156130
156132
156155
156156
156157
156163
156164
156165
156168
156173
156182
156190
156193
156199
156203
156216
156227
156233
156237
156241
156244
156253
156259
156262
156265
156278
156289
156329
156334
156341
156349
156355
156361
156369
156370
156371
156379
156386
156388
156390
156396
156400
156405
156413
156414
156420
156433
156437
156443
156448
156451
156460
156462

43520
43573
43616
43636
43642
43776
43838
43927
44104
44307
44414
44503
44910
45104
45116
45169
45172
45323
45328
45367
45393
45401
45484
45502
45566
45643
45672
45684
45761
45900
45908
45948
45973
45987
46075
46119
46153
46166
46168
46180
46366
46403
46463
46550
46606
46616
46653
46703
46822
46843
46874
46887
47094
47153
47226
47236
47277
47364
47375
47434
47474
47629
47705
47834
47863
47871
47897
47920
47939
47996
48000
48009
48027
48046
48067
48084
48142
48229
48374
48434
48585
48634
48662
48671
48689
48697
48726
48770
48772
48781
48818
48930
49039
49127
49163
49172
49279
49321
49331
49338
49357
49360
49373
49386
49408
49418
49548
49598
49656
49767
49811
49821
49946
49977
50023
50025
50036
50048
50098
50101
50104
50468
50516
50557
50587
50602
50604
50703
50712
50719
50732
50770
50935
50983
51008
51020
51045
51082
51218
51239
51298
51381
51395
51460
51483
51526
51579
51640
51749
51763
51832
51847
52062
52141
52147
52172
52283
52288
52398
52487
52554
52602
52627
52676
52682
52699
5270

594
599
603
613
616
618
634
636
642
652
661
671
683
693
696
738
739
743
749
757
764
766
769
784
795
816
817
821
825
826
831
836
842
844
846
852
875
876
877
879
884
890
894
907
912
913
985
990
993
1004
1009
1016
1026
1030
1040
1041
1044
1083
1115
1118
1134
1136
1139
1140
1149
1155
1159
1173
1188
1195
1221
1236
1253
1260
1279
1296
1305
1311
1315
1317
1326
1327
1365
1375
1381
1389
1402
1407
1411
1435
1462
1527
1538
1547
1549
1574
1579
1588
1595
1601
1607
1613
1619
1630
1636
1652
1657
1658
1664
1667
1672
1673
1674
1678
1682
1688
1692
1694
1698
1701
1734
1740
1758
1763
1770
1783
1785
1787
1803
1804
1805
1816
1821
1826
1838
1870
1872
1913
1916
1920
1929
1931
1949
1952
1967
2009
2021
2026
2029
2036
2047
2048
2054
2056
2061
2070
2102
2113
2148
2153
2156
2186
2197
2199
2212
2221
2238
2267
2299
2324
2348
2350
2355
2370
2371
2372
2378
2382
2404
2405
2408
2457
2463
2499
2515
2516
2539
2543
2548
2552
2573
2574
2598
2645
2646
2662
2669
2681
2693
2705
2752
2765
2769
2833
2848
2862
2875
2885
2896
2911

32496
32497
32498
32537
32579
32588
32589
32598
32600
32602
32607
32610
32620
32632
32643
32655
32671
32681
32685
32689
32702
32705
32710
32718
32721
32728
32735
32748
32774
32775
32779
32780
32782
32786
32788
32814
32815
32825
32841
32845
32857
32868
32871
32888
32889
32891
32948
32950
32953
32968
32992
33016
33026
33030
33032
33043
33058
33059
33076
33101
33106
33109
33118
33153
33170
33193
33200
33212
33228
33233
33256
33258
33269
33274
33289
33290
33300
33304
33307
33322
33328
33332
33333
33337
33365
33368
33372
33375
33377
33431
33434
33466
33472
33481
33482
33496
33536
33540
33559
33585
33650
33651
33693
33701
33704
33712
33717
33722
33762
33763
33779
33785
33786
33788
33791
33799
33829
33834
33837
33844
33855
33856
33877
33894
33897
33899
33903
33917
33918
33942
33946
33948
33954
33957
33961
33967
33968
33973
33982
33987
33992
33994
34016
34021
34025
34027
34034
34068
34081
34110
34124
34136
34139
34149
34164
34186
34202
34204
34206
34214
34236
34244
34246
34255
34261
34330
3433

KeyboardInterrupt: 

In [31]:
def get_tf_idf_bm25(int_document_id, query_term_id, document_term_freq):
    score = 0
    document_lenght=document_lengths[int_document_id]
    tf=np.log(1+(document_term_freq/document_lenght))
    idf=np.log(num_documents/len(inverted_index[query_term_id]))
    return tf,idf,document_lenght


def get_bm25_scores(query_id,doc_id):
    k1=1.2
    b=0.75
    query_tokens=tokenized_queries[query_id]    
    score=1  
    for query_token_id in query_tokens:
        count=0
        if(doc_id in inverted_index[query_token_id]):
            count=inverted_index[query_token_id][doc_id]
        tf,idf,document_lenght=get_tf_idf_bm25(doc_id, query_token_id,count)
        indv_score=(((k1+1)*tf)/(k1*((1-b)+b*(document_lenght/avg_doc_length))+tf))*idf
        score=score*indv_score
         

    return log(score)
bm_25_scores=get_bm25_scores('191',137864)    
print(bm_25_scores)        
    
    
    

0.0


In [36]:
def smoothing_jelenick_mercer(query_id,doc_id,lam):
    query_tokens=tokenized_queries[query_id]  
    doc_dict={}
    score=1
    for token in query_tokens:
        word_count=0
        if(doc_id in inverted_index[token]):
            word_count=inverted_index[token][doc_id]
        corpus_count=collection_frequencies[token]
        tf_corpus=np.log(1+(corpus_count/total_terms))
        tf,idf,doc_length=get_tf_idf_bm25(doc_id,token,word_count)
        score=score*(lam*(tf)+(1-lam)*tf_corpus)
    return score

        
smoothing_jelenick_mercer('191',137864,0.1)
    

1.5032210341918877e-18

In [39]:
def smoothing_dirchlet(query_id,doc_id,mu):
    query_tokens=tokenized_queries[query_id]  
    doc_dict={}
    score=1
    doc_len=document_lengths[doc_id]
    for token in query_tokens:
        word_count=0
        if(doc_id in inverted_index[token]):
            word_count=inverted_index[token][doc_id]
        corpus_count=collection_frequencies[token]
        tf_corpus=np.log(1+(corpus_count/total_terms))
        tf,idf,doc_length=get_tf_idf_bm25(doc_id,token,word_count)
        score=score*((doc_len/(doc_len+mu))*(tf)+(mu/(mu+doc_len))*tf_corpus)
    return score

        
smoothing_dirchlet('191',137864,500)

1.7875880609803646e-19

In [41]:
def absolute_discounting(query_id,doc_id,delta):
    query_tokens=tokenized_queries[query_id]  
    doc_dict={}
    score=1
    doc_len=document_lengths[doc_id]
    for token in query_tokens:
        word_count=0
        if(doc_id in inverted_index[token]):
            word_count=inverted_index[token][doc_id]
        corpus_count=collection_frequencies[token]
        tf_corpus=np.log(1+(corpus_count/total_terms))
        tf,idf,doc_length=get_tf_idf_bm25(doc_id,token,word_count)
        score=score*(max(tf-delta,0))+(delta*unique_terms_per_document[doc_id]*tf_corpus)/doc_len
    return score

        
absolute_discounting('191',137864,0.1)

5.0226178908037107e-07

In [22]:
def guassian_kernel(i,j,sigma):
    value=math.exp(-(i-j)/(2*sigma*sigma))
    return value
def triangle_kernel(i,j,sigma):
    value=0
    if(abs(i-j)<=sigma):
        value=1-(abs(i-j))/sigma
    return value
def cosine_kernel(i,j,sigma):
    value=0
    if(abs(i-j)<=sigma):
        value=0.5*(1+math.cos(abs(i-j)*pi/sigma))
    return value
def circle_kernel(i,j,sigma):
    value=0
    if(abs(i-j)<=sigma):
        value=math.sqrt(1-math.pow((i-j)/sigma,2))
    return value
def passge_kernel(i,j,sigma):
    value=0
    if(abs(i-j)<=sigma):
        value=1
    return value
def call_kernal(i,j,sigma,kernel_index):
    if(kernel_index==1):
        return guassian_kernel(i,j,sigma)
    if(kernel_index==2):
        return triangle_kernel(i,j,sigma)
    if(kernel_index==3):
        return cosine_kernel(i,j,sigma)
    if(kernel_index==4):
        return circle_kernel(i,j,sigma)
    if(kernel_index==5):
        return passge_kernel(i,j,sigma)
def create_word_vector(doc_token_ids):
    pos_word_vect={}
    dic={}
    le=len(doc_token_ids)
    for i in range(0,le):
        word_token=doc_token_ids[i]
        if(word_token not in dic):
            dic[word_token]=1
            word_token_indexes=[]
            for j in range(0,le):
                if(doc_token_ids[j]==word_token):
                    word_token_indexes.append(j)
            pos_word_vect[word_token]=word_token_indexes
    return pos_word_vect
    
def c_all_w_i(index,doc_token_ids,sigma,kernal_index,pos_word_vect):
    total_c=0
    le=len(doc_token_ids)
    for word in pos_word_vect.keys():
        word_token_indexes=pos_word_vect[word]
        for ind in word_token_indexes:
            calc=call_kernal(ind,index,sigma,kernal_index)
            total_c=total_c+calc
    return total_c
    
def plm(query_id,doc_id,kernel_index,index,mu,ext_doc_id):
    sigma=50
    query_tokens=tokenized_queries[query_id] 
    new_query_tokens=[]
    
    for token in query_tokens:
        if(token==0):
            rtt=1
        else:
            new_query_tokens.append(token)
    query_tokens=new_query_tokens
   
    ext_doc_id, doc_token_ids = index.document(doc_id)
    new_doc_token_ids=[]
    #print(len(doc_token_ids))
    for did in doc_token_ids:
        if(did is not 0):
            new_doc_token_ids.append(did)
    doc_token_ids=new_doc_token_ids
    #print(len(doc_token_ids))    
    score={}
    pos_word_dict=create_word_vector(doc_token_ids)
    le=len(doc_token_ids)
    for token in query_tokens:
        if(token in doc_token_ids):
            c_token_i=0
            word_index=pos_word_dict[token]
            for index in word_index:
                for i in range(0,le):
                    c_w_i=call_kernal(index,i,sigma,kernel_index)
                    total_c=c_all_w_i(i,doc_token_ids,sigma,kernel_index,pos_word_dict)
                    prob_c_w=c_w_i/total_c
                    if(token in score):
                        if(i in score[token]):
                            score[token][i]=prob_c_w+score[token][i]
                        else:
                            score[token][i]=prob_c_w
                    else:
                        score[token]={}
                        score[token][i]=prob_c_w
    le_query=len(query_tokens)
    query_dict={}
    for token in query_tokens:
        if(token in query_dict):
            val=query_dict[token]
            query_dict[token]=val+1
        else:
            query_dict[token]=1
    tot_query_sum=[]
    for i in range(0,le):
        c_token_i=0
        s_d_q_i=0
        for token in query_tokens:
            prob_token_query=query_dict[token]/le_query
            if(token in score):
                #print(token)
                c_token_i=score[token][i]
                #print(c_token_i)
            corpus_count=collection_frequencies[token]
            tf_corpus=np.log(1+(corpus_count/total_terms))
            #total_c=c_all_w_i(i,doc_token_ids,sigma,kernel_index)
            #c_w_i=(c_token_i+mu*tf_corpus)/(total_c+mu)
            c_w_i=(c_token_i+mu*tf_corpus)/(mu)
            s_d_q_i=s_d_q_i-prob_token_query*(math.log(prob_token_query/c_w_i))
           # print(s_d_q_i)
        tot_query_sum.append([i,s_d_q_i])                  
    tot_query_sum.sort(key=itemgetter(1), reverse=False)
    sc.ret_scores[ext_doc_id]=tot_query_sum[0][1]
    
run_retrieval('plm', plm)


Retrieving using plm
Queries completed 0
1966
Queries completed 1
23986


Process Process-16439:
Process Process-16528:
Process Process-15984:
Process Process-16532:
Process Process-16496:
Process Process-16563:
Process Process-16497:
Process Process-16494:
Process Process-16577:
Process Process-16571:
Process Process-16082:
Process Process-16560:
Process Process-16564:
Process Process-16566:
Process Process-16452:
Process Process-16543:
Process Process-16557:
Process Process-16409:
Process Process-16484:
Process Process-16498:
Traceback (most recent call last):
Process Process-16422:
Process Process-16558:
Traceback (most recent call last):
Process Process-16544:
Process Process-16561:
Process Process-16572:
Traceback (most recent call last):
Process Process-16575:
Process Process-16553:
Traceback (most recent call last):
Traceback (most recent call last):
Process Process-16373:
Process Process-16574:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/vikrant/anaconda3/lib/python3.6/multipr

  File "<ipython-input-22-c15fc170c813>", line 81, in plm
    pos_word_dict=create_word_vector(doc_token_ids)
  File "<ipython-input-22-c15fc170c813>", line 90, in plm
    total_c=c_all_w_i(i,doc_token_ids,sigma,kernel_index,pos_word_dict)
  File "/home/vikrant/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-22-c15fc170c813>", line 90, in plm
    total_c=c_all_w_i(i,doc_token_ids,sigma,kernel_index,pos_word_dict)
  File "/home/vikrant/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "<ipython-input-22-c15fc170c813>", line 90, in plm
    total_c=c_all_w_i(i,doc_token_ids,sigma,kernel_index,pos_word_dict)
  File "<ipython-input-22-c15fc170c813>", line 90, in plm
    total_c=c_all_w_i(i,doc_token_ids,sigma,kernel_index,pos_word_dict)
  File "<ipython-input-22-c15fc170c813>", line 57, in c_all_w_i
    total_c=total_c+calc
  File "<ipython-input-22-c15f

KeyboardInterrupt
KeyboardInterrupt
  File "<ipython-input-22-c15fc170c813>", line 57, in c_all_w_i
    total_c=total_c+calc
  File "<ipython-input-22-c15fc170c813>", line 2, in guassian_kernel
    value=math.exp(-(i-j)/(2*sigma*sigma))
  File "<ipython-input-22-c15fc170c813>", line 56, in c_all_w_i
    calc=call_kernal(ind,index,sigma,kernal_index)
  File "<ipython-input-22-c15fc170c813>", line 26, in call_kernal
    return guassian_kernel(i,j,sigma)
  File "<ipython-input-22-c15fc170c813>", line 56, in c_all_w_i
    calc=call_kernal(ind,index,sigma,kernal_index)
KeyboardInterrupt
  File "<ipython-input-22-c15fc170c813>", line 2, in guassian_kernel
    value=math.exp(-(i-j)/(2*sigma*sigma))
  File "<ipython-input-22-c15fc170c813>", line 2, in guassian_kernel
    value=math.exp(-(i-j)/(2*sigma*sigma))
  File "<ipython-input-22-c15fc170c813>", line 24, in call_kernal
    def call_kernal(i,j,sigma,kernel_index):
KeyboardInterrupt
KeyboardInterrupt
  File "<ipython-input-22-c15fc170c813>"

KeyboardInterrupt: 

KeyboardInterrupt
KeyboardInterrupt


### Task 2: Latent Semantic Models (LSMs) [15 points] ###

In this task you will experiment with applying distributional semantics methods ([LSI](http://lsa3.colorado.edu/papers/JASIS.lsi.90.pdf) **[5 points]** and [LDA](https://www.cs.princeton.edu/~blei/papers/BleiNgJordan2003.pdf) **[5 points]**) for retrieval.

You do not need to implement LSI or LDA on your own. Instead, you can use [gensim](http://radimrehurek.com/gensim/index.html). An example on how to integrate Pyndri with Gensim for word2vec can be found [here](https://github.com/cvangysel/pyndri/blob/master/examples/word2vec.py). For the remaining latent vector space models, you will need to implement connector classes (such as `IndriSentences`) by yourself.

In order to use a latent semantic model for retrieval, you need to:
   * build a representation of the query **q**,
   * build a representation of the document **d**,
   * calculate the similarity between **q** and **d** (e.g., cosine similarity, KL-divergence).
     
The exact implementation here depends on the latent semantic model you are using. 
   
Each of these LSMs come with various hyperparameters to tune. Make a choice on the parameters, and explicitly mention the reasons that led you to these decisions. You can use the validation set to optimize hyper parameters you see fit; motivate your decisions. In addition, mention clearly how the query/document representations were constructed for each LSM and explain your choices.

In this experiment, you will first obtain an initial top-1000 ranking for each query using TF-IDF in **Task 1**, and then re-rank the documents using the LSMs. Use TREC Eval to obtain the results and report on `NDCG@10`, Mean Average Precision (`MAP@1000`), `Precision@5` and `Recall@1000`.

Perform significance testing **[5 points]** (similar as in Task 1) in the class of semantic matching methods.

### Task 3:  Word embeddings for ranking [20 points] (open-ended) ###

First create word embeddings on the corpus we provided using [word2vec](http://arxiv.org/abs/1411.2738) -- [gensim implementation](https://radimrehurek.com/gensim/models/word2vec.html). You should extract the indexed documents using pyndri and provide them to gensim for training a model (see example [here](https://github.com/nickvosk/pyndri/blob/master/examples/word2vec.py)).
   
This is an open-ended task. It is left up you to decide how you will combine word embeddings to derive query and document representations. Note that since we provide the implementation for training word2vec, you will be graded based on your creativity on combining word embeddings for building query and document representations.

Note: If you want to experiment with pre-trained word embeddings on a different corpus, you can use the word embeddings we provide alongside the assignment (./data/reduced_vectors_google.txt.tar.gz). These are the [google word2vec word embeddings](https://code.google.com/archive/p/word2vec/), reduced to only the words that appear in the document collection we use in this assignment.

### Task 4: Learning to rank (LTR) [15 points] (open-ended) ###

In this task you will get an introduction into learning to rank for information retrieval.

You can explore different ways for devising features for the model. Obviously, you can use the retrieval methods you implemented in Task 1, Task 2 and Task 3 as features. Think about other features you can use (e.g. query/document length). Creativity on devising new features and providing motivation for them will be taken into account when grading.

For every query, first create a document candidate set using the top-1000 documents using TF-IDF, and subsequently compute features given a query and a document. Note that the feature values of different retrieval methods are likely to be distributed differently.

You are adviced to start some pointwise learning to rank algorithm e.g. logistic regression, implemented in [scikit-learn](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html).
Train your LTR model using 10-fold cross validation on the test set. More advanced learning to rank algorithms will be appreciated when grading.

### Task 4: Write a report [15 points; instant FAIL if not provided] ###

The report should be a PDF file created using the [sigconf ACM template](https://www.acm.org/publications/proceedings-template) and will determine a significant part of your grade.

   * It should explain what you have implemented, motivate your experiments and detail what you expect to learn from them. **[10 points]**
   * Lastly, provide a convincing analysis of your results and conclude the report accordingly. **[10 points]**
      * Do all methods perform similarly on all queries? Why?
      * Is there a single retrieval model that outperforms all other retrieval models (i.e., silver bullet)?
      * ...

**Hand in the report and your self-contained implementation source files.** Only send us the files that matter, organized in a well-documented zip/tgz file with clear instructions on how to reproduce your results. That is, we want to be able to regenerate all your results with minimal effort. You can assume that the index and ground-truth information is present in the same file structure as the one we have provided.
